# Regression

Nun kommen wir (endlich) zum eigentlichen Machine Learning

In [1]:
#import some necessary librairies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
warnings.warn_explicit = ignore_warn

from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
print(check_output(["ls", "../data/house-prices-advanced-regression-techniques"]).decode("utf8")) #check the files available in the directory

import pickle

all_data.pkl
data_description.txt
test.csv
test.pkl
test_1.pkl
train.csv
train.pkl
train_1.pkl
x_preprocessed.pkl
x_preprocessed_train.pkl
x_test.pkl
y_train.pkl


# Daten aus dem vorigen Schritt laden


In [2]:
with open('../data/house-prices-advanced-regression-techniques/x_preprocessed_train.pkl', 'rb') as handle:
    X_preprocessed_train = pickle.load(handle)

with open('../data/house-prices-advanced-regression-techniques/y_train.pkl', 'rb') as handle:
    y_train = pickle.load(handle)
    
with open('../data/house-prices-advanced-regression-techniques/x_test.pkl', 'rb') as handle:
    X_test = pickle.load(handle)


# Laden der Daten und Aufteilung in Train und Test Daten

Um die Güte eines trainierten Modells zu testen, werden sogenannte Test-Daten zurückgehalen. Diese Daten hat das Modell noch nie gesehen. So kann gemessen werden, wie gut das Netz generalisiert, also mit unbekannten Daten umgehen kann.

![image](../img/test_train_split.png)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_test = train_test_split(X_preprocessed_train, y_train, test_size=0.2, random_state=42)


In [3]:

# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'LinearRegression': {},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids = {}
for model_name, model in models.items():
    #print(f'Training and tuning {model_name}...')
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_score = np.sqrt(-1 * grids[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 968118361.7443278

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END max_depth=None, min_samples_split=5, n_estimators=500; total time=  10.8s
[CV] END max_depth=10, min_samples_split=10, n_estimators=500; total time=   7.9s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   2.4s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   1.9s
[CV] END max_depth=10, min_samples_split=2, n_estimators=500; total time=  11.6s
[CV] END max_depth=30, min_samples_split=5, n_estimators=500; total time=   7.0s
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time=   1.9s
[CV] END max_depth=None, min_samples_split=10, n_estimators=200; total time=   3.1s
[CV] END max_depth=10, min_samples_split=5, n_estimators=200; total time=   4.3s
[CV] END max_depth=10, min_samples_split=10, n_es

KeyboardInterrupt: 